# Lab 5: Loan Approval Prediction (Evaluable)

# Lab Goals:
In this project, we will develop and apply supervised classification methods to predict loan approval. 

We will follow the end-to-end **Machine Learning process** for a **financial risk use case**:
1.  **Data Understanding and Preparation**: Exploration of the dataset (EDA), feature engineering (missing values, categorical variables management).
2.  **Model Training**: Training a **Logistic Regression** model.
3.  **Model Evaluation & Business Insights**: Analysis of metrics (confusion matrix, precision, recall, density chart). We will analyze how to adjust the model's cut-off based on business goals and how to improve the model.

# Practice Information:

**Due date:** By end of October, 24th (23:59h)

**Submission procedure:** via Moodle.

**Name:** Luca Franceschi

**NIA:** 253885

## Part 0: Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

### Load Data

In [ ]:
df = pd.read_csv('loan_dataset.csv')

**Dataset description:**
- Loan_ID: A unique identification number for each loan application.
- Gender: The applicant's gender (e.g., Male, Female).
- Married: The applicant's marital status (e.g., Yes, No).
- Dependents: The number of dependents (e.g., children or other family members) the applicant has (e.g., 0, 1, 2, 3+).
- Education: The applicant's highest level of education (e.g., Graduate, Not Graduate).
- Self_Employed: Whether the applicant is self-employed (e.g., Yes, No).
- ApplicantIncome: The applicant's monthly income.
- CoapplicantIncome: The co-applicant's monthly income (if one exists, otherwise 0).
- LoanAmount: The total amount of the loan requested (in thousands).
- Loan_Amount_Term: The duration of the loan in months (e.g., 360 months = 30 years).
- Credit_History: A binary flag indicating if the applicant has a credit history that meets the bank's guidelines (1.0 = Yes, 0.0 = No).
- Property_Area: The type of area where the property is located (e.g., Urban, Semiurban, Rural).
- Loan_Status: (This is the target variable) The final decision on the loan application (Y = Approved, N = Rejected).

## Part 1: Exploratory Data Analysis (EDA)

### EX1: Initial Data Inspection

Get a first look at the data. Answer the questions:
- How many total rows (applicants) and columns (features) are in this dataset?
- Look at the Non-Null Count for all columns. Which columns have missing (null) values?
- What is the data type (Dtype) of the Credit_History column and the ApplicantIncome column?
- Look at the first 5 rows of data. What is our target variable (the column we want to predict), and what are its possible values?
- What is the average ApplicantIncome?
- Look at the min, max, and mean for the Credit_History column. What does this tell you about the data in this column?

In [ ]:
### YOUR CODE HERE
df.info()

- There are 614 rows and 13 columns.
- Gender, Married, Dependents, Self_Employed, LoanAmount, Loan_Amount_Term, Credit_History have null values.
- Credit_History is float64 and ApplicantIncome is int64

In [ ]:
df.head(5)

- The target variable is Loan_Status and its possible values are Y/N (binary outcome).

In [ ]:
print(f'Average applicant monthly income is {np.average(df['ApplicantIncome'].to_numpy()):.2f}')

In [ ]:
df.describe()

- The Credit_Score variable seems normalized to the range [0, 1] and is also very right-skewed. It also seems to be a binary variable.

### EX2: Analyze the Target Variable

Our target variable is `Loan_Status`. 
1.  Use `seaborn.countplot` to visualize the distribution of `Loan_Status`.
2.  Print the value counts the proportion.
3.  Is the dataset balanced or imbalanced?

In [ ]:
### YOUR CODE HERE
sns.countplot(df, x='Loan_Status', stat='proportion')
plt.show()

I would say it is not balanced.

### EX3: Visualize all features vs our target to understand the data

Let's understand the data and its relation to our target.

In order to do so, apply all we have learnt to to understand the numerical and categorical columns.

At the end answer the questions.

In [ ]:
### YOUR CODE HERE
df['Loan_Status'] = np.where(df['Loan_Status']=='Y', 1, 0)

In [ ]:
sns.heatmap(df.corr(numeric_only=True), annot=True)
plt.show()

- Based on the correlation output, which single numerical feature has the strongest (highest) correlation with Loan_Status?

The single numerical feature that has the strongest correlation with Loan_Status is **Credit_History**

- Look at the plot for Credit_History. Does having a Credit_History (value 1.0) seem to have a weak, moderate, or strong association with getting a loan approved?

It seems to have a moderate-to-strong positive correlation with getting a loan approved.

- In our previous exercise, we saw that Credit_History is missing 50 rows and Gender is missing 13 rows. Which of these missing values is "worse" or "scarier" for our model?

Credit_History missing values are worse than Gender missing ones since the correlation with our target variable is not that important for the Gender variable.

### EX4: Analyze Numerical Features

Let's see how `ApplicantIncome` relates to `Loan_Status`.

1.  Create a `seaborn.boxplot` with `x='Loan_Status'` and `y='ApplicantIncome'`.
2.  Give an insight about what you see.
3.  **Extra:** The plot is skewed by outliers. Create a new, filtered DataFrame and plot the boxplot again.

In [ ]:
### YOUR CODE HERE
sns.boxplot(df, x='Loan_Status', y='ApplicantIncome')
plt.show()

**Insight:** 

There does not seem to be a significant correlation between Loan_Status variables regarding to ApplicantIncome, since both boxplots are pretty much the same.

In [ ]:
incomes = df['ApplicantIncome'].to_numpy()
q1 = np.quantile(incomes, .25); q3 = np.quantile(incomes, .75)
iqr = q3-q1

df_f = df[(df['ApplicantIncome'] > q1-1.5*iqr) & (df['ApplicantIncome'] < q3+1.5*iqr)]

In [ ]:
sns.boxplot(df_f, x='Loan_Status', y='ApplicantIncome')
plt.show()

### EX5: Check for Missing Values

- How many missing values do we have in each feature?
- How many rows would we end up with if we dropped all rows with at least one missing value?

Use `.isnull().sum()` to find all columns with missing values and print the result.

In [ ]:
### YOUR CODE HERE
df.isnull().sum()

In [ ]:
df.dropna(how='any').info()

We would end up with 480 rows.

## Part 2: Data Preparation & Feature Engineering

### EX6: Handle Missing Values

We must clean the data before we can train a model.

Inputing values inserts noise to our dataframe since the inputation can be wrong, hence lying to our model.

QUESTION: Is it worth in this situation to impute values or would it be better to just drop the rows?

ANSWER: **IMO the best approach would be to impute the cells that belong to low-correlation features and drop the ones that belong to a high-correlation one. This way we do not risk to generate a ton of error while keeping values that we might be very important for the later analysis.**

### EX7: Understand, complete and execute the following code


### Handle Missing Values (Categorical)

It's best to impute missing values. For categorical features, we will use the **mode** (the most frequent value).

Steps:
1.  Create a list `cat_cols_with_na` with the names of the *categorical* columns that have missing values.
2.  Loop through this list and fill the `NA` values in `df` with the mode of each column.

In [ ]:
# We create a copy to avoid changing the original df
df_processed = df.copy()

In [ ]:
### FILL COLUMN LIST
cat_cols_with_na = ['Gender', 'Married', 'Dependents', 'Self_Employed']

for col in cat_cols_with_na:
    mode_val = df_processed[col].mode()[0]
    df_processed[col] = df_processed[col].fillna(mode_val)
    print(f"Filled NAs in '{col}' with mode: {mode_val}")

### Handle Missing Values (Numerical)

For numerical features, we will use the **median** (which is robust to outliers).

1.  Create a list `num_cols_with_na`.
2.  Loop through this list and fill the `NA` values in `df_processed` with the **median** of each column.

In [ ]:
### FILL COLUMN LIST
num_cols_with_na = ['LoanAmount', 'Loan_Amount_Term', 'Credit_History']

for col in num_cols_with_na:
    median_val = df_processed[col].median()
    df_processed[col] = df_processed[col].fillna(median_val)
    print(f"Filled NAs in '{col}' with median: {median_val}")

### EX8: Understand and execute the following code

Let's create a `TotalIncome` feature by combining `ApplicantIncome` and `CoapplicantIncome`.

1.  Create `df_processed['TotalIncome']` by adding the two columns.
2.  Drop the columns that we don't need for the model

In [ ]:
df_processed['TotalIncome'] = df_processed['ApplicantIncome'] + df_processed['CoapplicantIncome']

# Drop non needed columns for the model
columns_to_drop = ['ApplicantIncome', 'CoapplicantIncome', 'Loan_ID']
df_processed = df_processed.drop(columns_to_drop, axis=1)

print("Created 'TotalIncome' and dropped original columns.")
df_processed.head()

### EX9: Convert Target Variable

Machine learning models need numbers, not 'Y' and 'N'.

1.  Map `Loan_Status` to boolean (1 for 'Y' and 0 for 'N').

In [ ]:
### YOUR CODE HERE
# already done before
# df_processed['Loan_Status'] = ...
# df['Loan_Status'] = np.where(df['Loan_Status']=='Y', 1, 0)

## Part 3: Model Preparation (Split & Scale)

### Understand and execute the following code

1.  Create `X` by dropping `Loan_Status` from `df_final`.
2.  Create `y` with only the `Loan_Status` column.

In [ ]:
df_final = df_processed.copy()

In [ ]:
X = df_final.drop('Loan_Status', axis=1)
y = df_final['Loan_Status']

Split the Data

1.  `train_test_split` to create `X_train`, `X_test`, `y_train`, `y_test`.
2.  `test_size=0.2`, `random_state=42`, and **`stratify=y`** (important for imbalanced datasets).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

### One-Hot Encode Categorical Features

Models can't understand text like 'Male', 'Urban', etc. We need to convert them to dummy variables.

1.  `Credit_History` is 1.0/0.0, but should be treated as categorical. Convert it to `str` type: `df_processed['Credit_History'] = df_processed['Credit_History'].astype(str)`
2.  Get a list of all `object` type columns: `categorical_cols = df_processed.select_dtypes(include=['object']).columns`
3.  Use `pd.get_dummies()` on `df_processed`, passing the `categorical_cols` to the `columns` argument, and set `drop_first=True` (to avoid multicollinearity).
4.  Store the result in a new DataFrame `df_final`.



### Feature Scalling

Logistic Regression is sensitive to feature scales. We must scale our data as we did in previous labs.

In [ ]:
categorical_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
numeric_cols = ['LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'TotalIncome']


# Train SCALER (numeric columns only)
scaler = StandardScaler()
scaler.fit(X_train[numeric_cols])

# Train ENCODER (categoric columns only)
encoder = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)
encoder.fit(X_train[categorical_cols])

# Transform from the encoder trained
X_train_scaled_data = scaler.transform(X_train[numeric_cols])
df_train_scaled = pd.DataFrame(X_train_scaled_data, columns=numeric_cols, index=X_train.index)

X_train_encoded_data = encoder.transform(X_train[categorical_cols])
encoded_col_names = encoder.get_feature_names_out(categorical_cols)
df_train_encoded = pd.DataFrame(X_train_encoded_data, columns=encoded_col_names, index=X_train.index)

# Concat all features
X_train_processed_df = pd.concat([df_train_scaled, df_train_encoded], axis=1)

# Store the model columns
model_columns_ordered = X_train_processed_df.columns.tolist()

print(f"Número total de columnes del model: {len(model_columns_ordered)}")
print("Llista de columnes del model:", model_columns_ordered)
display(X_train_processed_df.head())

### IMPORTANT

The .fit() and .transform() steps for the scalers and encoders are done separately. This is because we .fit() them on our training data and then store these same fitted objects (e.g., in a pickle file) for our Streamlit app. The app will then use these fitted objects to .transform() new user input.

The .fit() step is done only on the train data (X_train) to prevent data leakage.

- Data Leakage is when information from your test set (which is supposed to be "unseen") leaks into your training process.

- If we .fit() the scaler on the entire dataset, it would learn the mean and standard deviation of the test data. If we fit the encoder on the entire dataset, it would learn all possible categories from the test data.

- This is "cheating." It gives our model information about the future, leading to falsely optimistic performance scores (like high accuracy) in our notebook.

- When the model goes live (in the Streamlit app) and sees truly new data, its performance will be much worse than we expected. By fitting only on X_train, we simulate this real-world scenario and get a true, reliable score.

## Part 4: Train the model

In [ ]:
def train_logistic_regression(X_train, y_train):
    """
    Train a Logistic Regression model on the training data.
    We use class_weight='balanced' to handle imbalance.
    """
    # Initialize the Logistic Regression model
    model = LogisticRegression(random_state=42, class_weight='balanced')
    
    # Train the model
    model.fit(X_train, y_train)
    
    return model

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def density_chart(data, label, class_names, model, cutoff=0.5):
    """
    Plots a density chart of the predicted probabilities for each class.
    """
    # Get the probabilities for each class
    probabilities = model.predict_proba(data)
    
    # Get the probability of the positive class (class 1)
    prob_positive = probabilities[:, 1]
    
    # Create a DataFrame for plotting
    df_plot = pd.DataFrame({'Probability': prob_positive, 'Label': label})
    
    # Plot the density chart
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df_plot, x='Probability', hue='Label', common_norm=False, fill=True, palette='coolwarm', legend=class_names)
    
    # Customize the plot
    plt.title('Density Chart of Predicted Probabilities', fontsize=16)
    plt.xlabel('Predicted Probability', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.axvline(x=cutoff, color='red', linestyle='--', label=f'Cut-off ({cutoff})')
    # plt.legend(title='Actual Label', labels=class_names)
    plt.show()

def evaluate_model_at_cutoff(model, X_test, y_test, cutoff=0.5):
    """
    Evaluates a model at a specific probability cutoff.
    
    It calculates predictions, prints a classification report,
    and calls the 'density_chart' and 'plot_confusion_matrix' functions.
    """
    class_names=['Rejected (0)', 'Approved (1)']
    
    # 1. Get probability of positive class (class 1)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # 2. Generate predictions based on the custom cutoff
    y_pred = (y_prob >= cutoff).astype(int)
    
    print(f"      EVALUATION REPORT AT CUT-OFF = {cutoff}      ")
    
    # 3. Print the classification report
    print("\n--- Classification Report ---")
    print(classification_report(y_test, y_pred, target_names=class_names))
    
    # 4. Call the density_chart function
    # Aquesta funció ja crea la seva pròpia figura
    print("\n--- Probability Density Plot ---")
    density_chart(data=X_test, 
                  label=y_test, 
                  class_names=class_names, 
                  model=model, 
                  cutoff=cutoff)
    
    # 5. Calculate and plot the confusion matrix
    print(f"\n--- Confusion Matrix (at Cut-off = {cutoff}) ---")
    
    # Calculate the cm
    cm = confusion_matrix(y_test, y_pred)
    
    # Create a new figure for the confusion matrix
    plt.figure(figsize=(6, 5))
    plot_confusion_matrix(cm=cm, 
                          classes=class_names, 
                          title=f'Confusion Matrix (Cut-off = {cutoff})')
    plt.show()

### EX10: Execute - Train Logistic Regression

1.  Call the `train_logistic_regression` function with your scaled training data to create `model_lr`.

In [ ]:
### your code here
model_lr = train_logistic_regression(X_train_processed_df, y_train)
print(model_lr)

### Model evaluation - Predict the test set

First, we need to encode and scale the data as we did with the train set:

In [ ]:
# Scale numeric cols
X_test_scaled_data = scaler.transform(X_test[numeric_cols])

df_test_scaled = pd.DataFrame(X_test_scaled_data, 
                            columns=numeric_cols, 
                            index=X_test.index) # Mantenim l'índex per al concat

# Encode categorical cols
X_test_encoded_data = encoder.transform(X_test[categorical_cols])

encoded_col_names = encoder.get_feature_names_out(categorical_cols)
df_test_encoded = pd.DataFrame(X_test_encoded_data, 
                             columns=encoded_col_names, 
                             index=X_test.index) # Mantenim l'índex per al concat

# Concat both dataframes horizontally
X_test_processed_df = pd.concat([df_test_scaled, df_test_encoded], axis=1)

# Reorder cols (we need the same order as in training)
X_test_processed_df = X_test_processed_df[model_columns_ordered]

print("Dades de X_test processades correctament.")
display(X_test_processed_df.head())

### EX11: Evaluate Logistic Regression

1.  Call the function evaluate_model_at_cutoff() with the requested params
2.  Question: Which are the main steps of this function?

Answer:

As specified in the function, the steps are:

1. Get probability of positive class (class 1, loan is given)
2. Generate predictions based on the cutoff (above cutoff, class 1 selected; below, class 0)
3. Print report, density charts and confusion matrix of the prediction

In [ ]:
### your code here
evaluate_model_at_cutoff(model_lr, X_test_processed_df, y_test, cutoff=0.5)

### EX12: Interpret Logistic Regression Coefficients

A huge advantage of Logistic Regression is its interpretability.

1.  Create a `pd.DataFrame` to store the `model_lr.coef_[0]` and the `X.columns`.
2.  Sort the DataFrame by the coefficient value (descending) and display it.
3.  Which feature is the most important positive predictor? Which is a strong negative predictor?

In [ ]:
coefficients = pd.DataFrame({
    'Feature': model_columns_ordered,
    'Coefficient': model_lr.coef_[0]
})

coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(data=coefficients, x='Coefficient', y='Feature', hue='Feature', palette='vlag', legend=False)
plt.title('Feature Importance (Coefficients) from Logistic Regression')
plt.show()

display(coefficients)

**Answer:** 

The most important positive predictor is the Credit_History, while a strong negative predictor is Education_Not_Graduate (strongest indeed).

### EX13: Business Scenario 1 (Conservative Bank)

**Goal:** Be very conservative and **minimize risk**. We want to avoid approving bad loans.

1.  Which are the 'bad loans'? TP, FP, TN, FN?
2.  What do we want to maximize here? (answer with a metric)
3.  What do we have to do with the cutoff to increase the answer 2 metric?
4.  Play with the following cell, define the best cutoff (aprox) and analyze: What happened to Recall? What happened to Precision?

**Answers:**

1. Bad loans are False Positives (FP). They are predicted to be able to pay the loan, but they will default.
2. We want to maximize the precision since we, as a conservative bank, cannot stand loan defaults. Mathematically speaking,

$$
\lim_{FP \to 0} \frac{TP}{TP + FP} = 1
$$

3. We should increase the cutoff, which will make it more difficult for undecided predictions to pass the cutoff.

**NOTE:** We need to take into account that the rate at which FP decrease w.r.t. TP is also important for this analysis. In this case, if the threshold is moved from 0.5 to 0.6 the precision actually decreases. This question is quite complex and cannot be answered easily.

In [ ]:
### YOUR CODE HERE
new_threshold_recall = 0.7

evaluate_model_at_cutoff(model_lr, X_test_processed_df, y_test, cutoff=new_threshold_recall)

### EX14: Business Scenario 2 (Aggressive Growth)

**Goal:** Aggressive growth. We want to **capture as many good customers as possible**. We are willing to accept a little more risk to avoid missing opportunities.

1.  What do we want to maximize here? (answer with a metric and justify your answer)
2.  What do we have to do with the cutoff to increase the answer 1 metric?
3.  Play with the following cell, define the best cutoff (aprox) and analyze: What happened to Recall? What happened to Precision?

**Answers**

1. In this scenario we want to maximize the recall, since we can tolerate a higher amount of FP. We want to get people more loans and we do not care that much if some of them default.
2. We can decrease the cutoff. This way, undecided predictions will have a higher change of being accepted.

In [ ]:
### YOUR CODE HERE
new_threshold_recall = 0.3

evaluate_model_at_cutoff(model_lr, X_test_processed_df, y_test, cutoff=new_threshold_recall)

Lowering the threshold has improven the recall significantly.

### EX15: Our current models are relatively straightforward. Briefly describe two or three techniques you would try next to improve the model's performance.

Answer: I would look into optimizing some LogisticRegression hyperparameters, try out other tree-based models such as hierarchical clustering methods, and i would also take a more in-depth look at how the cutoff behaves, maybe even trying to optimize it through a grid-search approach.

### EX16: Looking at our EDA and model performance, what do you believe is the single biggest factor limiting our model's accuracy?

Answer: Probably the lack of more rows of data and/or features.

### EX17: Consider two new applicants. Based on your interpretation of the model's coefficients (from Exercise 6.2), which applicant is more likely to be approved, and why?
- Applicant A: Has a Credit_History = 1.0, is Married, and lives in a Semiurban area.
- Applicant B: Has a Credit_History = 0.0, is Not Graduate, and is Self_Employed

Answer:

Applicant A would be more likely to be approved since he has all the positively correlated variables in his favor, while Applicant B is pretty much the opposite.

## PART 5. Communicating results through a Visual Analytics web tool.

We are ready to share our results with our Consultant team and customers. To do it, we will create a Data product using the Streamlit framework. This web app should have a least 2 tabs: one for data exploration and other for salary prediction.

Before building this app, you should store the ML model and labencoders in a **pickle** format as follows. This pickle file will be the one you need to call in your web app.

In [ ]:
import pickle

In [ ]:
data = {
    "model": model_lr,
    "encoder": encoder,
    "scaler": scaler,
    "numeric_cols": numeric_cols,
    "categorical_cols": categorical_cols,
    "model_columns_ordered": model_columns_ordered,
}
with open('loan_app_artifacts.pkl', 'wb') as file:
    pickle.dump(data, file)

You can verify the pickle file works as follows: first, open the file storing into 3 variables the regressor, the labencoder for country and the labencoder for education.

In [ ]:
with open('loan_app_artifacts.pkl', 'rb') as file:
    data = pickle.load(file)

model_loaded = data["model"]
encoder_loaded = data["encoder"]
scaler_loaded = data["scaler"]
numeric_cols_loaded = data["numeric_cols"]
categorical_cols_loaded = data["categorical_cols"]
model_columns_ordered_loaded = data["model_columns_ordered"]

You can also make a prediction with a sample as follows:

In [ ]:
raw_columns = ['LoanAmount', 'Loan_Amount_Term', 'TotalIncome', 
               'Gender', 'Married', 'Dependents', 
               'Education', 'Self_Employed', 'Credit_History', 'Property_Area']

sample_data = [[
    150.0,       # LoanAmount
    360.0,       # Loan_Amount_Term
    8000.0,      # TotalIncome
    'Male',      # Gender
    'Yes',       # Married
    '0',         # Dependents (recorda que era un 'object', per això va com a string)
    'Graduate',  # Education
    'No',        # Self_Employed
    1.0,         # Credit_History (float, 1.0 o 0.0)
    'Semiurban'  # Property_Area
]]

sample_df = pd.DataFrame(sample_data, columns=raw_columns)
display(sample_df)

In [ ]:
# Normalize as in training

# Scale numeric cols
sample_df_scaled_data = scaler.transform(sample_df[numeric_cols_loaded])

df_sample_scaled = pd.DataFrame(sample_df_scaled_data, 
                            columns=numeric_cols_loaded, 
                            index=sample_df.index) # Mantenim l'índex per al concat

# Encode categorical cols
sample_df_encoded_data = encoder.transform(sample_df[categorical_cols_loaded])

encoded_col_names = encoder.get_feature_names_out(categorical_cols_loaded)
df_sample_encoded = pd.DataFrame(sample_df_encoded_data, 
                             columns=encoded_col_names, 
                             index=sample_df.index) # Mantenim l'índex per al concat

# Concat both dataframes horizontally
sample_df_processed_df = pd.concat([df_sample_scaled, df_sample_encoded], axis=1)

# Reorder cols (we need the same order as in training)
sample_df_processed_df = sample_df_processed_df[model_columns_ordered]

print("Dades de sample_df processades correctament.")
display(sample_df_processed_df.head())

In [ ]:
MY_CUTOFF = 0.5

# Predict with our cutoff
(model_loaded.predict_proba(sample_df_processed_df)[:, 1] >= MY_CUTOFF).astype(int)

Now, you are ready to go to next step: building the Visual Analytics tool based on Streamlit. Recall, your app should have a main .py file (for instance, **my_app.py**) and a folder called **pages** with all .py files you want to show in your app.